In [ ]:
print("hola")

In [1]:
from pathlib import Path

from twitter_trends import ArgsTwitterTrends, ParamsTwitterTrends, WOEIDCountry, requests_and_process
from scraping_kit import BotScraper, load_db_and_bots

path_data = Path("data")
db_tw, bots = load_db_and_bots(path_data, "scrape_tw")

Collection Names: ['trends', 'raw', 'topics']
Bots: bots=[BotScraper(acc_name='Vera')]


In [2]:
from scraping_kit.db.models.topics import Topic
topics: list[Topic] = []
for doc in db_tw.coll.topics.find():
    topic = Topic(**doc)
    topics.append(topic)


In [3]:
import json
with open("CLASSES_TWITTER.json", "r") as f:
    CLASSES_TWITTER = json.load(f)

In [8]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")



Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBartForSequenceClassification: ['model.encoder.version', 'model.decoder.version']
- This IS expected if you are initializing TFBartForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBartForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBartForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBartForSequenceClassification for predictions without further training.


In [13]:
topics[0]

Topic(trend_name='Christmas', topic_classes=TopicClasses(text='the 3 Kings arrive to Bethlehem on the 6th of January to see baby Jesus . we put some wine and cookies for the Kings\nher chip had been registered to t.co/GsmqJ9qosQ . she was left for dead in\nmalcy had to take him out of the box about a MILLION TIMES . he did everything he could\nLiving room when the Christmas tree comes down https://t.co/wTLcYjYh7a\nThis thread will cover some interesting points from a recent interview I read concerning the celebration of Christmas in Japan. 1/9 https://t.co/enIh5T7SZG', labels=['technology', 'entertainment', 'news', 'food', 'travel', 'arts & culture', 'careers', 'hobbies & interests', 'gaming', 'science', 'music', 'business & finance', 'outdoors', 'sports', 'movies and tv', 'fashion & beauty'], scores=[0.6827396154403687, 0.5170348286628723, 0.4710201323032379, 0.36188793182373047, 0.2930907607078552, 0.27791041135787964, 0.12259001284837723, 0.1096777468919754, 0.10653416812419891, 0.0

In [19]:
threshold = 0.5
text = topics[0].topic_classes.text
labels, scores = topics[0].get_labels_scores_thresh(threshold)
labels

['technology', 'entertainment']

In [20]:
print(len(labels))
TOPICS2 = []
for label in labels:
    TOPICS2.extend(CLASSES_TWITTER[label])
TOPICS2

2


['Technology',
 'Augmented reality',
 'Cloud computing',
 'Cloud platforms',
 'Computer programming',
 'Cryptocurrencies',
 'Data science',
 'Databases',
 'Drone technology',
 'FinTech',
 'Information security',
 'Internet of things',
 'Tech brands',
 'Tech news',
 'Tech personalities',
 'Virtual reality',
 'Entertainment',
 'Celebrities',
 'Comedy',
 'Digital creators',
 'Entertainment brands',
 'Popular franchises',
 'Theater']

In [21]:
asd = classifier(
    text,
    candidate_labels = TOPICS2,
    multi_label = True
)
asd

{'sequence': 'the 3 Kings arrive to Bethlehem on the 6th of January to see baby Jesus . we put some wine and cookies for the Kings\nher chip had been registered to t.co/GsmqJ9qosQ . she was left for dead in\nmalcy had to take him out of the box about a MILLION TIMES . he did everything he could\nLiving room when the Christmas tree comes down https://t.co/wTLcYjYh7a\nThis thread will cover some interesting points from a recent interview I read concerning the celebration of Christmas in Japan. 1/9 https://t.co/enIh5T7SZG',
 'labels': ['Internet of things',
  'Digital creators',
  'Tech personalities',
  'Entertainment',
  'Tech brands',
  'Entertainment brands',
  'Technology',
  'Information security',
  'Tech news',
  'Popular franchises',
  'Comedy',
  'Augmented reality',
  'Cloud platforms',
  'FinTech',
  'Computer programming',
  'Theater',
  'Data science',
  'Cloud computing',
  'Databases',
  'Virtual reality',
  'Drone technology',
  'Celebrities',
  'Cryptocurrencies'],
 'sco

In [ ]:
import json
with open("asd.json", "w") as f:
    json.dump(topics, f, default=lambda obj: obj.isoformat())